In [9]:
import pandas as pd
from pprint import pprint

FORMULA_NUMBER = 1
NUM_BRACKETS = 100
YEAR = 2018

data_file = 'config\TeamData.csv'
teams_df = pd.read_csv(data_file)

# This formula defines the power rating for each team
def formula(team, win, loss, pct, RPI, Kenpom, BPI):
    power = pct * (Kenpom/35) * (BPI) * (1/(30+RPI))
    return power

# Generate power rating for each team
power_ratings = dict()
for index, team in teams_df.iterrows():
    name = team['Team']
    power = formula(team['Team'],team['Win'],team['Loss'],team['Pct'],team['RPI'],team['Kenpom'],team['BPI'])
    power_ratings.update({name:power})

# Returns the probability that team1 beats team2
def prob(team1, team2):
    return team1 / (team1+team2) * 100

# First we'll set up the empty bracket
empty_bracket_file = 'config\EmptyBracket.csv'
empty_bracket_df = pd.read_csv(empty_bracket_file,dtype='object')

# Make a copy for manipulation
bracket_df = empty_bracket_df.copy()

import random

def predict_rd(rd):
    next_rd_teams = []
    for team in rd:
        team1 = team
        team2 = next(rd)
        # print(team1 + ' - ' + team2 + ': ' + str(prob(power_ratings[team1],power_ratings[team2])) + ' %')
        odds = prob(power_ratings[team1],power_ratings[team2])
        rand = random.uniform(0,100)
        if rand < odds:
            winner = team1
        else:
            winner = team2  
        next_rd_teams.append(winner)   
        # print(odds, rand, winner)
    return next_rd_teams

def predict_bracket():
    Rd1 = iter(empty_bracket_df['Rd1'].values)
    Rd2_teams = predict_rd(Rd1)
    # print(Rd2_teams)
    Rd2 = iter(Rd2_teams)
    S16_teams = predict_rd(Rd2)
    # print(S16_teams)
    S16 = iter(S16_teams)
    E8_teams = predict_rd(S16)
    # print(E8_teams)
    E8 = iter(E8_teams)
    FF_teams = predict_rd(E8)
    # print(FF_teams)
    FF = iter(FF_teams)
    C_teams = predict_rd(FF)
    # print(C_teams)
    C = iter(C_teams)
    W = predict_rd(C)
    winner = ''.join(W)
    # print(winner)
    return Rd2_teams, S16_teams, E8_teams, FF_teams, C_teams, winner

# Now we will fill out the bracket, preparing it for writing to csv
def fill_bracket(bracket_df):
    bracket = predict_bracket()
    
    Rd2 = bracket[0]
    S16 = bracket[1]
    E8 = bracket[2]
    FF = bracket[3]
    C = bracket[4]
    winner = bracket[5]

    i = 0
    for team in Rd2:
        bracket_df.at[i,'Rd2'] = team
        i+=2
    i = 0
    for team in S16:
        bracket_df.at[i,'S16'] = team
        i+=4
    i = 0
    for team in E8:
        bracket_df.at[i,'E8'] = team
        i+=8
    i = 0
    for team in FF:
        bracket_df.at[i,'FF'] = team
        i+=16
    i = 0
    for team in C:
        bracket_df.at[i,'C'] = team
        i+=32
    bracket_df.at[0,'W'] = winner
    
    return bracket_df

bracket_df = fill_bracket(bracket_df)

for i in range(1,NUM_BRACKETS+1):
    filename = r'C:\Users\Brian\Documents\MarchMadness2019\brackets\formula' + str(FORMULA_NUMBER) + '_' + str(YEAR) + '_' + str(i) + '.csv'
    bracket_df = fill_bracket(bracket_df)
    write_csv_error = bracket_df.to_csv (filename, index = None, header=True)

if not write_csv_error:
    print('Successfully completed')

Successfully completed
